# HW2- Binary Classification 
- Detravious Jamari Brinkley
- CSCI-544: Applied Natural Language Processing
- python version: 3.11.4

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

import sklearn
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Pipeline
1. Read data
2. Keep reviews and ratings
3. Create binary and ternary classes
4. Clean data steps
4. Clean data function
5. Split data
6. Load pretrained model and train my model
    - Get similarity for the pretrained model
    - Get similarity for my trained model
9. Extract word embeddings
    - Get word embeddings for pretrained model
    - Get word embeddings for my model
10. Simple models
    - Get accuracy for perceptron on pretrained model
    - Get accuracy for svm on pretrained model
    - Get accuracy for perceptron on my model
    - Get accuracy for svm on my model
11. What do I conclude from comparing performances
12. Feedforward Neural Network

## Read Data

In [2]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
amazon_reviews_copy_df = pd.read_csv(dataset, sep='\t', on_bad_lines='skip', low_memory=False)

## Keep Reviews and Ratings

In [3]:
reviews_ratings_df = amazon_reviews_copy_df.loc[0:, ['star_rating', 'review_body']]
reviews_ratings_df.reset_index(drop=True)

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [4]:
def generate_sample_reviews(df: pd.DataFrame, review_col_name: str, number_of_reviews: int = 3):
    """Include reviews and ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    review_col_name: `str`
        The specific_column to get the reviews and ratings of
    
    number_of_reviews: `int`
        Number of samples to include


    Return
    ------
    Nothing; instead, print the reviews with ratings
    """


    columns_to_include = [review_col_name, 'star_rating']

    # Initialize an empty list to store dictionaries
    list_of_dicts = []

    # Iterate over the specified columns and retrieve the first three rows
    for row in df[columns_to_include].head(3).to_dict(orient='records'):
        list_of_dicts.append({'star_rating': row['star_rating'], review_col_name: row[review_col_name]})

    for dictionary in list_of_dicts:
        print(dictionary)

 ## Create binary and ternary classes


In [5]:
def update_data_type(df: pd.DataFrame, col_name: str):
    """Update the data type of the star ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with rating values

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened

    """

    valid_ratings = ['1','2','3','4','5']
    star_rating_series = df[col_name].copy()

    # Convert type to strings
    star_rating_series.astype('str')

    # Check valid list and see which of our stars match
    rows = star_rating_series.index
    is_rating_in_valid_ratings = rows[star_rating_series.isin(valid_ratings)]

    # Convert to list
    is_rating_in_valid_ratings = is_rating_in_valid_ratings.to_list()

    updated_df = df.iloc[is_rating_in_valid_ratings]
    updated_df[col_name] = updated_df[col_name].astype(int)
    return updated_df

In [6]:
updated_reviews_ratings_df = update_data_type(reviews_ratings_df, 'star_rating')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_78622/2907883124.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_df[col_name] = updated_df[col_name].astype(int)


In [7]:
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [8]:
updated_reviews_ratings_df = updated_reviews_ratings_df.dropna()
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [9]:
# Check for NaN values
nan_check = updated_reviews_ratings_df.isna()

# Display the DataFrame with True where NaN values exist
print(nan_check)

# Check if any NaN value exists in the DataFrame
if nan_check.any().any():
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

         star_rating  review_body
0              False        False
1              False        False
2              False        False
3              False        False
4              False        False
...              ...          ...
2640249        False        False
2640250        False        False
2640251        False        False
2640252        False        False
2640253        False        False

[2640080 rows x 2 columns]
There are no NaN values in the DataFrame.


In [10]:
print("# reviews per rating", updated_reviews_ratings_df['star_rating'].value_counts())

# reviews per rating star_rating
5    1582704
4     418348
1     306967
3     193680
2     138381
Name: count, dtype: int64


In [11]:
def sample_star_ratings(df: pd.DataFrame, col_name: str, star_value: int, number_of_reviews: int):
    """Build a subset balanced dataset with reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The dataframe to use
    col_name: `str`
        The name of the column to get reviews from
    star_value: `int`
        The star rating of the review
    number_of_reviews: `int`
        The number of sub reviews to include in sample

    Return
    ------
    rating_df, sampled_rating_df: `tuple`
        All reviews with that rating and the subset reviews with that rating
    """
    
    rating_df = df[df[col_name] == star_value]
    sampled_rating_df = rating_df.sample(n=number_of_reviews)
    return rating_df, sampled_rating_df

In [12]:
# subset_reviews = 50000
subset_reviews = 300

one_star = 1
rating_one, rating_one_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', one_star, subset_reviews)
two_stars = 2
rating_two, rating_two_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', two_stars, subset_reviews)
three_stars = 3
rating_three, rating_three_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', three_stars, subset_reviews)
four_stars = 4
rating_four, rating_four_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', four_stars, subset_reviews)
five_stars = 5
rating_five, rating_five_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', five_stars, subset_reviews)

In [13]:
sampled_reviews_df = pd.concat([rating_one_sampled, rating_two_sampled, rating_three_sampled, rating_four_sampled, rating_five_sampled])

In [14]:
sampled_reviews_df

,star_rating,review_body
525783,1,feeble and fragile ... good design idea... inf...
880086,1,Years ago we had an HP printer that created al...
2208773,1,"In less than one month of use, the yellow tone..."
1015475,1,I have no luck getting any support (human) to ...
1833772,1,Although it didn't slip right into place like ...
...,...,...
1461079,5,Not sure how I've been teaching for 4 years wi...
1603521,5,"like it alot, holds up well on the outside of ..."
1232082,5,Very happy with the bubble wrap and speedy del...
2582464,5,I had no problems at all with this printer. A...


In [15]:

def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):
    """Categorizes reviews by adding a rating

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    rating_col: `str`
        Column with rating values
    
    threshold: `int`
        Where to split the ratings such that categories can be formed

    sentiment_type: `str`
        One of three types of sentiment: positive, negative, or neural

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened
    """


    if sentiment_type == 'positive_review_class':
        positive_review_threshold = df[rating_col].astype('int32') > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'negative_review_class':
        negative_review_threshold = df[rating_col].astype('int32') < threshold
        df = df[negative_review_threshold]
        df[sentiment_type] = 2

    elif sentiment_type == 'neutral_review_class':
        neutral_review_threshold = df[rating_col].astype('int32') == threshold
        df = df[neutral_review_threshold]
        df[sentiment_type] = 3
        
    return df

In [16]:
negative_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'negative_review_class')
neutral_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'neutral_review_class')
positive_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'positive_review_class')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_78622/637639312.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 2
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_78622/637639312.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 3
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_78622/637639312.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [17]:
sampled_reviews_ratings_df = pd.concat([negative_review_class_df, neutral_review_class_df, positive_review_class_df])
sampled_reviews_ratings_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class
525783,1,feeble and fragile ... good design idea... inf...,2.0,NaN,NaN
880086,1,Years ago we had an HP printer that created al...,2.0,NaN,NaN
2208773,1,"In less than one month of use, the yellow tone...",2.0,NaN,NaN
1015475,1,I have no luck getting any support (human) to ...,2.0,NaN,NaN
1833772,1,Although it didn't slip right into place like ...,2.0,NaN,NaN
...,...,...,...,...,...
1461079,5,Not sure how I've been teaching for 4 years wi...,NaN,NaN,1.0
1603521,5,"like it alot, holds up well on the outside of ...",NaN,NaN,1.0
1232082,5,Very happy with the bubble wrap and speedy del...,NaN,NaN,1.0
2582464,5,I had no problems at all with this printer. A...,NaN,NaN,1.0


In [18]:
negative_reviews_df = sampled_reviews_ratings_df['negative_review_class'].dropna()
neutral_reviews_df = sampled_reviews_ratings_df['neutral_review_class'].dropna()
positive_reviews_df = sampled_reviews_ratings_df['positive_review_class'].dropna()

In [19]:
sampled_reviews_ratings_df['binary_review_class'] = pd.concat([negative_reviews_df, positive_reviews_df])
sampled_reviews_ratings_df['ternary_review_class'] = pd.concat([negative_reviews_df, neutral_reviews_df, positive_reviews_df])

In [20]:
sampled_reviews_ratings_df['binary_review_class'].unique()

array([ 2., nan,  1.])

## Clean data steps

### Lower case

In [21]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        # NOT all reviews are strings, thus all can't be converted to lower cased
        if type(text_review) != str:
            print(True, text_review)
            converted_str = str(text_review)
            lower_case_reviews.append(text_review)
         
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)

    updated_df['lower_cased'] = lower_case_reviews
    return updated_df

In [22]:
# reviews_lower_cased = convert_reviews_to_lower_case(sampled_reviews_ratings_df, 'review_body')

In [23]:
# reviews_lower_cased

In [24]:
# print("reviews_lower_cased:")
# generate_sample_reviews(reviews_lower_cased, 'lower_cased', 3)

### Remove HTML and URLs

In [25]:
def remove_html_and_urls(df: pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    # url_pattern = re.compile(r'https?://\S+|www\. \S+')

    cleaned_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            # Check and remove HTML tags
            has_html = bool(re.search('<.*?>', text_review))
            if has_html == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass

            no_html_review = re.sub('<.*?>', ' ', text_review)
            # print("Review", text_reviews_idx, "without HTML -- ", no_html_review)
        
            # Check and remove URLs
            has_url = bool(re.search(r'http\S+', no_html_review))
            if has_url == True:
                # print("Review", text_reviews_idx, "has URL --", no_html_review)
                pass

            no_html_url_review = re.sub(r'http\S+', '', no_html_review)
            # print("Review", text_reviews_idx, "without HTML, URL -- ", no_html_url_review)
            # print()
            cleaned_reviews.append(no_html_url_review)
        else:
            # print(text_reviews_idx, text_review)
            cleaned_reviews.append(text_review)
            

    updated_df['without_html_urls'] = cleaned_reviews
    return updated_df

In [26]:
# no_html_urls_df = remove_html_and_urls(reviews_lower_cased, 'lower_cased')

In [27]:
# no_html_urls_df

In [28]:
# print("without_html_urls:")
# generate_sample_reviews(no_html_urls_df, 'without_html_urls', 3)

### Remove Contractions

In [29]:
store_contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they're": "they are",
    "wasn't": "was not",
    "we're": "we are",
    "weren't": "were not",
    "won't": "will not",
    "wouldn't": "would not",
    "you're": "you are",
    "you'll": "you will",
    "you'd": "you would",
    "we'll": "we will",
    "we've": "we have",
    "we'd": "we would",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "I'd": "I would",
    "it'll": "it will",
    "they'll": "they will",
    "they've": "they have",
    "they'd": "they would",
    "he'll": "he will",
    "he'd": "he would",
    "she'll": "she will",
    "we'd": "we would",
    "we'll": "we will",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "I'll": "I will",
    "I'd": "I would",
    "it's": "it is",
    "it'd": "it would",
    "i'm": "I am",
    "he's": "he is",
    "he'll": "he will",
    "she's": "she is",
    "she'll": "she will",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "who's": "who is",
    "who'll": "who will",
    "who'd": "who would",
    "what's": "what is",
    "what'll": "what will",
    "what'd": "what would",
    "when's": "when is",
    "when'll": "when will",
    "when'd": "when would",
    "where's": "where is",
    "where'll": "where will",
    "where'd": "where would",
    "why's": "why is",
    "why'll": "why will",
    "why'd": "why would",
    "how's": "how is",
    "how'll": "how will",
    "how'd": "how would"
}


In [30]:
def locate_and_replace_contractions(review):
    """Find the contractions to replace from a specific review

    Parameters
    ----------
    review: `str`
        A specific review

    Return
    ------
    non_contraction_review: `str`
        The updated specific review with contractions expanded
    
    """
    if isinstance(review, str):
        get_words = review.split()

        store_non_contraction_words = []

        for word in get_words:
            if word in store_contractions:
                non_contraction_form = store_contractions[word]
                # print(word, "-->", non_contraction_form)

                store_non_contraction_words.append(non_contraction_form)

            else:
                # print(word)
                store_non_contraction_words.append(word)

        non_contraction_review = ' '.join(store_non_contraction_words)
        return non_contraction_review
    else:
        return review


In [31]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_contractions_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        # print("Review", text_reviews_idx, "with possible contraction(s) -- ", text_review)

        without_contraction = locate_and_replace_contractions(text_review)

        # print("Review", text_reviews_idx, "without contraction -- ", without_contraction)
        # print()

        without_contractions_reviews.append(without_contraction)

    updated_df['without_contractions'] = without_contractions_reviews
    return updated_df

In [32]:
# no_contractions_df = remove_contractions(no_html_urls_df, 'without_html_urls')

In [33]:
# no_contractions_df

In [34]:
# print("without_contractions:")
# generate_sample_reviews(no_contractions_df, 'without_contractions', 3)

### Remove Non-alphabetical characters

In [35]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """

    alphabetical_char_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        
        if isinstance(text_review, str):

            # Check for non-alphabetical characters
            has_non_alphabetical_char = bool(re.search(r'[^a-zA-Z]', text_review))
            if has_non_alphabetical_char == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass
            
            # Remove non-alphabetical characters
            with_alphabetical_char = re.sub(r'[^a-zA-Z\s]', ' ', text_review)
            # print("Review", text_reviews_idx, "has HTML -- ", with_alphabetical_char)
            alphabetical_char_reviews.append(with_alphabetical_char)
        else:
            alphabetical_char_reviews.append(text_review)

    updated_df['with_alpha_chars_only'] = alphabetical_char_reviews
    return updated_df

In [36]:
# only_alpha_chars_df = remove_non_alphabetical_characters(no_contractions_df, 'without_contractions')

In [37]:
# only_alpha_chars_df

In [38]:
# print("with_alpha_chars_only:")
# generate_sample_reviews(only_alpha_chars_df, 'with_alpha_chars_only', 3)

### Remove extra spaces

In [39]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    single_spaced_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
        # Check if there are any extra spaces
            has_extra_space = bool(re.search(r' +', text_review))
            if has_extra_space == True:
                # print("Review", text_reviews_idx, "has extra space -- ", text_review)
                pass
            
            # Remove extra spaces
            single_spaced_review = re.sub(r' +', ' ', text_review)
            # print("Review", text_reviews_idx, "without extra space -- ", single_spaced_review)
            # print()
            
            single_spaced_reviews.append(single_spaced_review)
        else:
            single_spaced_reviews.append(text_review)

    updated_df['without_extra_space'] = single_spaced_reviews
    return updated_df

In [40]:
# no_extra_space_df = remove_extra_spaces(only_alpha_chars_df, 'with_alpha_chars_only')

In [41]:
# no_extra_space_df

In [42]:
# print("without_extra_space:")
# generate_sample_reviews(no_extra_space_df, 'without_extra_space', 3)

### Remove the stop words 

In [43]:
def filter_stop_words(df:pd.DataFrame, col_name: str):
    """Filter stop words out from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_stop_words_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    stop_words = set(stopwords.words("english"))

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            text_review_words = word_tokenize(text_review) 

        

            # print("Before stop word removal", text_reviews_idx, " -- ", text_review)

            filtered_review = []

            for text_review_words_idx in range(len(text_review_words)):
                text_review_word = text_review_words[text_review_words_idx]
                
                # Check if review word is a stop word
                if text_review_word in stop_words:
                    # print("  Stop word -- ", text_review_word)
                    pass
                else:
                    # print(text_review_word, " -- is NOT a stop word in review")
                    filtered_review.append(text_review_word)

            
            filtered_review = " ".join(filtered_review)
            # print("After stop word removal", text_reviews_idx, " -- ", filtered_review)
            # print()
            
            without_stop_words_reviews.append(filtered_review)
        else:
            without_stop_words_reviews.append(text_review)
        

    updated_df['without_stop_words'] = without_stop_words_reviews
    return updated_df

In [44]:
# no_stop_words_df = filter_stop_words(no_extra_space_df, 'without_extra_space')

In [45]:
# no_stop_words_df

In [46]:
# print("without_stop_words:")
# generate_sample_reviews(no_stop_words_df, 'without_stop_words', 3)

### Perform lemmatization  

- "A sentence with many words"
    - "words" -> word

In [47]:
def lemmentize_review(df:pd.DataFrame, col_name: str):
    """Lemmentize all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    lemmed_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    lem = WordNetLemmatizer()

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]   
        if isinstance(text_review, str):     
            words_in_review = word_tokenize(text_review) 

            # print("Before lem update", text_reviews_idx, " -- ", text_review)
            # print("Lemmed words", words_in_review)
            

            lemmed_sentence = []

            # Split review into words
            for lemmed_words_idx in range(len(words_in_review)):
                word = words_in_review[lemmed_words_idx]
                
                apply_lemmatization = lem.lemmatize(word)
                # print(apply_lemmatization)
                
                lemmed_sentence.append(apply_lemmatization)
                filtered_review = " ".join(lemmed_sentence)
        
            # print("After lem update -- ", filtered_review)
            # print()

            lemmed_reviews.append(filtered_review)
        else:
            lemmed_reviews.append(text_review)

    updated_df['lemmed_reviews'] = lemmed_reviews
    return updated_df

In [48]:
# lemmed_df = lemmentize_review(no_stop_words_df, 'without_stop_words')

In [49]:
# lemmed_df

In [50]:
# print("without_unlemmed_words:")
# generate_sample_reviews(lemmed_df, 'lemmed_reviews', 3)

## Clean data function

In [51]:
def preprocess_data(df, col_name):
    """Perform lower case, remove HTML and URLs, remove contractions, remove non-alphabetical characters, remove extra spaces, remove stop words, and lemmatize"""

    print("original reviews:")
    generate_sample_reviews(df, col_name, 3)

    reviews_lower_cased = convert_reviews_to_lower_case(df, col_name)
    print("reviews_lower_cased:")
    generate_sample_reviews(reviews_lower_cased, 'lower_cased', 3)

    no_html_urls_df = remove_html_and_urls(reviews_lower_cased, 'lower_cased')
    print("without_html_urls:")
    generate_sample_reviews(no_html_urls_df, 'without_html_urls', 3)

    no_contractions_df = remove_contractions(no_html_urls_df, 'without_html_urls')
    print("without_contractions:")
    generate_sample_reviews(no_contractions_df, 'without_contractions', 3)

    only_alpha_chars_df = remove_non_alphabetical_characters(no_contractions_df, 'without_contractions')
    print("with_alpha_chars_only:")
    generate_sample_reviews(only_alpha_chars_df, 'with_alpha_chars_only', 3)

    no_extra_space_df = remove_extra_spaces(only_alpha_chars_df, 'with_alpha_chars_only')
    print("without_extra_space:")
    generate_sample_reviews(no_extra_space_df, 'without_extra_space', 3)

    no_stop_words_df = filter_stop_words(no_extra_space_df, 'without_extra_space')
    print("without_stop_words:")
    generate_sample_reviews(no_stop_words_df, 'without_stop_words', 3)
    
    lemmed_df = lemmentize_review(no_stop_words_df, 'without_stop_words')
    print("without_unlemmed_words:")
    generate_sample_reviews(lemmed_df, 'lemmed_reviews', 3)

    return lemmed_df


In [52]:
cleaned_reviews_df = preprocess_data(sampled_reviews_ratings_df, 'review_body')

original reviews:
{'star_rating': 1, 'review_body': 'feeble and fragile ... good design idea... inferior joint materials ... three dimensional joints w/ soft plastic ...'}
{'star_rating': 1, 'review_body': 'Years ago we had an HP printer that created all kinds of havoc with Windows.  Now we have the HP Officejet 4630 and it too is creating all kinds of havoc.  Finally have learned our lesson, NO MORE HP PRINTERS.  Worked fine for few months and now only light blue printing, colors strong but no black print.  Did all the troubleshooting, re-installed, etc etc and now have drop kicked it out the door.'}
{'star_rating': 1, 'review_body': 'In less than one month of use, the yellow toner exploded inside of our machine. There is yellow toner dust everywhere in and around out printer! This is not acceptable. The blue has began to drip blue dust as well inside the printer.'}
reviews_lower_cased:
{'star_rating': 1, 'lower_cased': 'feeble and fragile ... good design idea... inferior joint materi

In [53]:
cleaned_reviews_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,binary_review_class,ternary_review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words,lemmed_reviews
525783,1,feeble and fragile ... good design idea... inf...,2.0,NaN,NaN,2.0,2.0,feeble and fragile ... good design idea... inf...,feeble and fragile ... good design idea... inf...,feeble and fragile ... good design idea... inf...,feeble and fragile good design idea inf...,feeble and fragile good design idea inferior j...,feeble fragile good design idea inferior joint...,feeble fragile good design idea inferior joint...
880086,1,Years ago we had an HP printer that created al...,2.0,NaN,NaN,2.0,2.0,years ago we had an hp printer that created al...,years ago we had an hp printer that created al...,years ago we had an hp printer that created al...,years ago we had an hp printer that created al...,years ago we had an hp printer that created al...,years ago hp printer created kinds havoc windo...,year ago hp printer created kind havoc window ...
2208773,1,"In less than one month of use, the yellow tone...",2.0,NaN,NaN,2.0,2.0,"in less than one month of use, the yellow tone...","in less than one month of use, the yellow tone...","in less than one month of use, the yellow tone...",in less than one month of use the yellow tone...,in less than one month of use the yellow toner...,less one month use yellow toner exploded insid...,le one month use yellow toner exploded inside ...
1015475,1,I have no luck getting any support (human) to ...,2.0,NaN,NaN,2.0,2.0,i have no luck getting any support (human) to ...,i have no luck getting any support (human) to ...,i have no luck getting any support (human) to ...,i have no luck getting any support human to ...,i have no luck getting any support human to he...,luck getting support human help setting thing ...,luck getting support human help setting thing ...
1833772,1,Although it didn't slip right into place like ...,2.0,NaN,NaN,2.0,2.0,although it didn't slip right into place like ...,although it didn't slip right into place like ...,although it did not slip right into place like...,although it did not slip right into place like...,although it did not slip right into place like...,although slip right place like old one fit pro...,although slip right place like old one fit pro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461079,5,Not sure how I've been teaching for 4 years wi...,NaN,NaN,1.0,1.0,1.0,not sure how i've been teaching for 4 years wi...,not sure how i've been teaching for 4 years wi...,not sure how I have been teaching for 4 years ...,not sure how I have been teaching for years ...,not sure how I have been teaching for years wi...,sure I teaching years without one even redicul...,sure I teaching year without one even rediculo...
1603521,5,"like it alot, holds up well on the outside of ...",NaN,NaN,1.0,1.0,1.0,"like it alot, holds up well on the outside of ...","like it alot, holds up well on the outside of ...","like it alot, holds up well on the outside of ...",like it alot holds up well on the outside of ...,like it alot holds up well on the outside of y...,like alot holds well outside door small packag...,like alot hold well outside door small package...
1232082,5,Very happy with the bubble wrap and speedy del...,NaN,NaN,1.0,1.0,1.0,very happy with the bubble wrap and speedy del...,very happy with the bubble wrap and speedy del...,very happy with the bubble wrap and speedy del...,very happy with the bubble wrap and speedy del...,very happy with the bubble wrap and speedy del...,happy bubble wrap speedy delivery,happy bubble wrap speedy delivery
2582464,5,I had no problems at all with this printer. A...,NaN,NaN,1.0,1.0,1.0,i had no problems at all with this printer. a...,i had no problems at all with this printer. a...,i had no problems at all with this printer. af...,i had no problems at all with this printer af.

## Split data

In [54]:
def split_data(df, review_class):
    embeddings_df = df.dropna(subset=[review_class])
    print(len(embeddings_df), embeddings_df['star_rating'].unique())

    specific_review_class = embeddings_df[review_class]
    print(specific_review_class.unique())

    text = embeddings_df.loc[:, ['lemmed_reviews']]
    print(text)

    ### Train test split so I can have the same train
    X_train, X_test, y_train, y_test = train_test_split(text, specific_review_class, test_size=0.2, random_state=42)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    return X_train, X_test, y_train, y_test

In [55]:
print("Binary\n")
binary_X_train, binary_X_test, binary_y_train, binary_y_test = split_data(cleaned_reviews_df, 'binary_review_class')
print("\nTernary")
ternary_X_train, ternary_X_test, ternary_y_train, ternary_y_test = split_data(cleaned_reviews_df, 'ternary_review_class')

Binary

1200 [1 2 4 5]
[2. 1.]
                                            lemmed_reviews
525783   feeble fragile good design idea inferior joint...
880086   year ago hp printer created kind havoc window ...
2208773  le one month use yellow toner exploded inside ...
1015475  luck getting support human help setting thing ...
1833772  although slip right place like old one fit pro...
...                                                    ...
1461079  sure I teaching year without one even rediculo...
1603521  like alot hold well outside door small package...
1232082                  happy bubble wrap speedy delivery
2582464  problem printer receiving problem setting inst...
1361289  time come new standard home phone fit requirem...

[1200 rows x 1 columns]
(960, 1) (240, 1) (960,) (240,)

Ternary
1500 [1 2 3 4 5]
[2. 3. 1.]
                                            lemmed_reviews
525783   feeble fragile good design idea inferior joint...
880086   year ago hp printer created kind havoc w

## Load pretrained model and train my model

In [56]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim.downloader as api
pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

2024-02-07 19:35:54,880 : INFO : loading projection weights from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2024-02-07 19:36:32,131 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-02-07T19:36:32.131496', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [57]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __init__(self, df: pd.DataFrame, col_name: str):
        self.df = df
        self.col_name = col_name

    def __iter__(self):
        """

        Parameters
        ----------
        df: `pd.DataFrame`
            The data
        
        col_name: `str`
            Column with reviews

        words_in_model: `list`
            Words in Word2Vec model

        Return
        ------
        
        """

        text_reviews = self.df[self.col_name].values

        for text_reviews_idx in range(len(text_reviews)):
            text_review = text_reviews[text_reviews_idx]
            # print(text_reviews_idx, "--", text_review)

            yield utils.simple_preprocess(text_review)
        

2024-02-07 19:36:32,176 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-07 19:36:32,177 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2024-02-07 19:36:32,178 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2024-02-07T19:36:32.178237', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'created'}


In [58]:
import gensim.models

In [59]:
print("Binary Case")
binary_X_train_sentences = MyCorpus(binary_X_train, 'lemmed_reviews')
my_binary_X_train_model = gensim.models.Word2Vec(sentences=binary_X_train_sentences, vector_size=300, window=11, min_count=10)

# X test - get embeddings from my_binary_X_train_model -- vec_king = my_binary_X_train_model.wv['king']
# binary_X_test_sentences = MyCorpus(binary_X_test, 'lemmed_reviews')
# sentences = MyCorpus(sampled_reviews_ratings_df, 'review_body')
# print("\nSentences", binary_X_test_sentences)

print("\nTernary Case")
ternary_X_train_sentences = MyCorpus(ternary_X_train, 'lemmed_reviews')
my_ternary_X_train_model = gensim.models.Word2Vec(sentences=ternary_X_train_sentences, vector_size=300, window=11, min_count=10)


2024-02-07 19:36:32,268 : INFO : collecting all words and their counts
2024-02-07 19:36:32,269 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-02-07 19:36:32,309 : INFO : collected 4553 word types from a corpus of 27418 raw words and 960 sentences
2024-02-07 19:36:32,309 : INFO : Creating a fresh vocabulary
2024-02-07 19:36:32,312 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 577 unique words (12.67% of original 4553, drops 3976)', 'datetime': '2024-02-07T19:36:32.312229', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2024-02-07 19:36:32,312 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 18480 word corpus (67.40% of original 27418, drops 8938)', 'datetime': '2024-02-07T19:36:32.312962', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (cla

Binary Case


2024-02-07 19:36:32,646 : INFO : EPOCH 1: training on 27418 raw words (15060 effective words) took 0.3s, 57163 effective words/s
2024-02-07 19:36:32,711 : INFO : EPOCH 2: training on 27418 raw words (15149 effective words) took 0.1s, 258254 effective words/s
2024-02-07 19:36:32,772 : INFO : EPOCH 3: training on 27418 raw words (15087 effective words) took 0.1s, 284447 effective words/s
2024-02-07 19:36:32,823 : INFO : EPOCH 4: training on 27418 raw words (15178 effective words) took 0.0s, 353232 effective words/s
2024-02-07 19:36:32,823 : INFO : Word2Vec lifecycle event {'msg': 'training on 137090 raw words (75581 effective words) took 0.5s, 151728 effective words/s', 'datetime': '2024-02-07T19:36:32.823837', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'train'}
2024-02-07 19:36:32,824 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=577, vector_size=300, alp


Ternary Case


2024-02-07 19:36:33,079 : INFO : EPOCH 2: training on 33838 raw words (20043 effective words) took 0.1s, 359382 effective words/s
2024-02-07 19:36:33,140 : INFO : EPOCH 3: training on 33838 raw words (20025 effective words) took 0.1s, 374098 effective words/s
2024-02-07 19:36:33,203 : INFO : EPOCH 4: training on 33838 raw words (20030 effective words) took 0.1s, 362047 effective words/s
2024-02-07 19:36:33,204 : INFO : Word2Vec lifecycle event {'msg': 'training on 169190 raw words (100189 effective words) took 0.3s, 319608 effective words/s', 'datetime': '2024-02-07T19:36:33.204614', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'train'}
2024-02-07 19:36:33,205 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=694, vector_size=300, alpha=0.025>', 'datetime': '2024-02-07T19:36:33.205121', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 

### Similar scores

[ ] Write summary of differences between their model and my model

In [60]:
result = pretrained_word_two_vec_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7118193507194519)]


In [61]:
my_trained_binary_X_train_model = my_binary_X_train_model.wv
my_trained_binary_X_train_model

my_trained_ternary_X_train_model = my_ternary_X_train_model.wv
my_trained_ternary_X_train_model

In [62]:
# TODO: Fix with proper exs
my_result = my_trained_binary_X_train_model.most_similar(positive=['started', 'even'], negative=['plus', 'toner'], topn=10)
print(my_result)


[('jammed', 0.007604660931974649), ('level', 0.007585110608488321), ('cord', 0.00752635532990098), ('exactly', 0.006755036301910877), ('add', 0.006493216380476952), ('useful', 0.006399194709956646), ('dollar', 0.006285232026129961), ('easier', 0.006052954122424126), ('drop', 0.005979087669402361), ('smooth', 0.005930942017585039)]


## Extract word embeddings

In [63]:
def word_embeddings(df: pd.DataFrame, col_name: str, model_to_use):
    """Extract word embeddings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    model_to_use:
        Either the pretrained model or my pretrained model

    Return
    ------
    
    """

    sentence_vectorized = []
    mean_sentences_vectorized = []
    sentences = df[col_name].values

    for sentences_idx in range(len(sentences)):
        vectorized_words = []
        sentence = sentences[sentences_idx]
        # print("Sentence", sentences_idx)
        # print("Sentence", sentences_idx, "Pre-vectorized -- ", sentence)
        for word in sentence.split(" "):
            if word in model_to_use.key_to_index:
                vector_of_word = model_to_use[word]
                vectorized_words.append(vector_of_word)
                # print("--->", word, "is in model with vector lenght of", len(vector_of_word))
            else:
                vector_of_word = np.random.rand(model_to_use.vector_size)
                vectorized_words.append(vector_of_word)
                
        sentence_vectorized.append(vectorized_words)
        # print("Sentence", sentences_idx, "Post-vectorized \n")

        mean_of_sentence = np.mean(sentence_vectorized[sentences_idx], axis=0)
        # print(len(mean_of_sentence), mean_of_sentence)
        mean_sentences_vectorized.append(mean_of_sentence)


    return mean_sentences_vectorized

In [64]:
# binary_X_train

### Extract embeddings for pretrained model

In [65]:
print("Binary Pretrained Train")
pretrained_binary_train_embeddings = word_embeddings(binary_X_train, 'lemmed_reviews', pretrained_word_two_vec_model)
pretrained_binary_train_embeddings = np.array(pretrained_binary_train_embeddings)

print("Binary Pretrained Test")
pretrained_binary_test_embeddings = word_embeddings(binary_X_test, 'lemmed_reviews', pretrained_word_two_vec_model)
pretrained_binary_test_embeddings = np.array(pretrained_binary_test_embeddings)

pretrained_binary_train_embeddings.shape, pretrained_binary_test_embeddings.shape

Binary Pretrained Train
Binary Pretrained Test


((960, 300), (240, 300))

In [66]:
print("Ternary Pretrained Train")
pretrained_ternary_train_embeddings = word_embeddings(ternary_X_train, 'lemmed_reviews', pretrained_word_two_vec_model)
pretrained_ternary_train_embeddings = np.array(pretrained_ternary_train_embeddings)

print("Ternary Pretrained Test")
pretrained_ternary_test_embeddings = word_embeddings(ternary_X_test, 'lemmed_reviews', pretrained_word_two_vec_model)
pretrained_ternary_test_embeddings = np.array(pretrained_ternary_test_embeddings)

pretrained_ternary_train_embeddings.shape, pretrained_ternary_test_embeddings.shape

Ternary Pretrained Train
Ternary Pretrained Test


((1200, 300), (300, 300))

### Extract embeddings for my model

In [67]:
print("Binary My Model Train")
my_binary_train_embeddings = word_embeddings(binary_X_train, 'lemmed_reviews', my_trained_binary_X_train_model)
my_binary_train_embeddings = np.array(my_binary_train_embeddings)

print("Binary My Model Test")
my_binary_test_embeddings = word_embeddings(binary_X_test, 'lemmed_reviews', my_trained_binary_X_train_model)
my_binary_test_embeddings = np.array(my_binary_test_embeddings)

my_binary_train_embeddings.shape, my_binary_test_embeddings.shape

Binary My Model Train
Binary My Model Test


((960, 300), (240, 300))

In [68]:
print("Ternary My Model Train")
my_ternary_train_embeddings = word_embeddings(ternary_X_train, 'lemmed_reviews', my_trained_ternary_X_train_model)
my_ternary_train_embeddings = np.array(my_ternary_train_embeddings)

print("Ternary My Model Test")
my_ternary_test_embeddings = word_embeddings(ternary_X_test, 'lemmed_reviews', my_trained_ternary_X_train_model)
my_ternary_test_embeddings = np.array(my_ternary_test_embeddings)

my_ternary_train_embeddings.shape, my_ternary_test_embeddings.shape

Ternary My Model Train
Ternary My Model Test


((1200, 300), (300, 300))

In [69]:
def tf_idf_feature_extraction(df: pd.DataFrame, col_name: str):
    """Extract the TF-IDF features from the reviews.

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    tf_idf_features:
        A matrix containing the TF-IDF features extracted
        
    """
    
    vectorizer = TfidfVectorizer()
    tf_idf_features = vectorizer.fit_transform(df[col_name])

    return tf_idf_features

In [70]:
# tf_idf_features = tf_idf_feature_extraction(lemmed_df, 'lemmed_reviews')


In [71]:
# tf_idf_features

## Simple models

In [72]:
def eval_accuracy(y_true, y_prediction):
    return sklearn.metrics.accuracy_score(y_true, y_prediction)

# def eval_precision(y_true, y_prediction):
#     return sklearn.metrics.precision_score(y_true, y_prediction)

# def eval_recall(y_true, y_prediction):
#     return sklearn.metrics.recall_score(y_true, y_prediction)

# def eval_f1_score(y_true, y_prediction):
#     return sklearn.metrics.f1_score(y_true, y_prediction)

In [73]:
def train_eval_metric(y_train_true, y_train_predictions):
    accuracy = eval_accuracy(y_train_true, y_train_predictions)
    # precision = eval_precision(y_train_true, y_train_predictions)
    # recall = eval_recall(y_train_true, y_train_predictions)
    # f1 = eval_f1_score(y_train_true, y_train_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        # 'Precision': precision,
        # 'Recall': recall,
        # 'F1 Score': f1
    }

    return metrics_dict

def test_eval_metric(y_test_true, y_test_predictions):
    accuracy = eval_accuracy(y_test_true, y_test_predictions)
    # precision = eval_precision(y_test_true, y_test_predictions)
    # recall = eval_recall(y_test_true, y_test_predictions)
    # f1 = eval_f1_score(y_test_true, y_test_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        # 'Precision': precision,
        # 'Recall': recall,
        # 'F1 Score': f1
    }

    return metrics_dict

### Get accuracy for perceptron on pretrained model

In [74]:
def perceptron_model(X_train, X_test, y_train, y_test): 

    technique = Perceptron(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [75]:
my_binary_train_embeddings.shape, my_binary_test_embeddings.shape, binary_y_train.shape, binary_y_test.shape

((960, 300), (240, 300), (960,), (240,))

In [76]:
# Pretrained model
perceptron_train_metrics, perceptron_test_metrics = perceptron_model(pretrained_binary_train_embeddings, pretrained_binary_test_embeddings, binary_y_train, binary_y_test)

In [77]:
perceptron_train_metrics, perceptron_test_metrics

({'Accuracy': 0.86875}, {'Accuracy': 0.7291666666666666})

### Get accuracy for perceptron on my model

In [78]:
# My model
my_perceptron_train_metrics, my_perceptron_test_metrics = perceptron_model(my_binary_train_embeddings, my_binary_test_embeddings, binary_y_train, binary_y_test)
 

In [79]:
my_perceptron_train_metrics, my_perceptron_test_metrics

({'Accuracy': 0.675}, {'Accuracy': 0.45416666666666666})

In [80]:
def svm_model(X_train, X_test, y_train, y_test): 

    technique = LinearSVC(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


### Get accuracy for svm on pretrained model

In [81]:
svm_train_metrics, svm_test_metrics = svm_model(pretrained_binary_train_embeddings, pretrained_binary_test_embeddings, binary_y_train, binary_y_test)

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [82]:
svm_train_metrics, svm_test_metrics

({'Accuracy': 0.8645833333333334}, {'Accuracy': 0.7416666666666667})

### Get accuracy for svm on pretrained model

In [83]:
my_svm_train_metrics, my_svm_test_metrics = svm_model(my_binary_train_embeddings, my_binary_test_embeddings, binary_y_train, binary_y_test)

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [84]:
my_svm_train_metrics, my_svm_test_metrics

({'Accuracy': 0.7333333333333333}, {'Accuracy': 0.5416666666666666})

## What do I conclude from comparing performances

## Feedforward Neural Network
- MLP for sentiment analysis classification
- 2 hidden layers each with 50 and 10 nodes, respectively
- Cross entropy loss
- I decide other hyperparameters (ie: nonlinearity, #epochs, etc)

### Binary Classification

In [85]:
# binary_review_class = binary_embeddings_df['binary_review_class']
# binary_review_class.unique()

In [86]:
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

class Net(nn.Module):
    """Define the NN architecture"""

    def __init__(self, num_h1_nodes: int, num_h2_nodes: int, d: int, num_output_classes: int, dropout_rate: float):
        super(Net, self).__init__()

        # self.height = heightght = height
        # self.width = width
        # linear layer (1200 x 300 dot 300 x 50 -> 1200 x 50)
        self.fc1 = nn.Linear(d, num_h1_nodes)
        # linear layer (1200 x 50 dot 50 x 10 -> 1200 x 10)
        self.fc2 = nn.Linear(num_h1_nodes, num_h2_nodes)
        # linear layer (1200 x 50 dot 50 x 10 -> 1200 x 10)
        self.fc3 = nn.Linear(num_h2_nodes, num_output_classes) # change to 3 for ternery
        # dropout to prevent overfitting
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        
        # add hidden layer, with relu activation function, dropout, relu, dropout, output
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x
    
    def train_network(self, number_of_epochs: int, optimizer, criterion_function, train_loader):
        # set initial "min" to infinity
        valid_loss_min = np.Inf

        for epoch in range(number_of_epochs):
            train_loss = 0.0
            

            ###################
            # train the model #
            ###################
            self.train() # prep model for training
            for data, target in train_loader:
                # clear the gradients of all optimized variables
                optimizer.zero_grad()
                # forward pass to compute predictions, loss, backward pass to compute gradient wrt model params
                output = self(data)
                loss = criterion_function(output, target)
                loss.backward()
                optimizer.step()
                # update running training loss
                train_loss += loss.item() * data.size(0)
            
            # print training statistics 
            # calculate average loss over an epoch
            train_loss = train_loss / len(train_loader.dataset)
            
            print('Epoch: {} \tTraining Loss: {:.6f}'.format(
                epoch+1, 
                train_loss,
                ))
            
            # # save model if validation loss has decreased
            # if train_loss <= valid_loss_min:
            #     print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            #     valid_loss_min,
            #     train_loss))
            #     torch.save(self.state_dict(), 'nn_model.pt')
            #     valid_loss_min = train_loss

    def predict(self, data_loader):
        predictions = []
        ground_truth = []

        for i, (inputs, targets) in enumerate(data_loader):
            outputs = self(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            ground_truth.extend(targets.cpu().numpy())

        # Convert predictions and ground truth lists to numpy arrays
        predictions = np.array(predictions)
        ground_truth = np.array(ground_truth)

        # Calculate accuracy
        accuracy = (predictions == ground_truth).mean()
        print("Accuracy:", accuracy)

        return accuracy

In [87]:
N_binary_embeddings, d_binary_embeddings = my_binary_train_embeddings.shape

In [88]:
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 2
dropout_rate = 0.2

net_model = Net(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)
print(net_model)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [89]:
def train_binary_model(binary_train_embeddings, binary_train_y):
    binary_train_y = binary_train_y.replace(2, 0)
    data_tensor = torch.tensor(binary_train_embeddings, dtype=torch.float32)
    target_tensor = torch.tensor(binary_train_y.values, dtype=torch.long)
    dataset = TensorDataset(data_tensor, target_tensor)

    # how many samples per batch to load
    batch_size = 64

    # as a positive integer will turn on multi-process data loading with the specified number of loader worker processes; otherwise, single-process data loading
    num_workers = 0

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

    number_of_epochs = 3
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net_model.parameters(), lr=0.01)
    output_of_model = net_model.train_network(number_of_epochs, optimizer, criterion, train_loader)
    
    return output_of_model

In [90]:
binary_y_train

1391287    2.0
237246     2.0
832872     2.0
1691509    1.0
2257654    2.0
          ... 
693447     1.0
1385083    1.0
1610920    1.0
1978013    1.0
2254291    1.0
Name: binary_review_class, Length: 960, dtype: float64

In [91]:
train_binary_model(pretrained_binary_train_embeddings, binary_y_train)

Epoch: 1 	Training Loss: 0.707500
Epoch: 2 	Training Loss: 0.705389
Epoch: 3 	Training Loss: 0.703271


In [92]:
train_binary_model(my_binary_train_embeddings, binary_y_train)

Epoch: 1 	Training Loss: 0.700519
Epoch: 2 	Training Loss: 0.700768
Epoch: 3 	Training Loss: 0.698298


In [93]:
def predict_binary_model(binary_test_embeddings, binary_test_y):
    binary_test_y = binary_test_y.replace(2, 0)
    target_array = binary_test_y.values
    data_tensor = torch.tensor(binary_test_embeddings, dtype=torch.float32)


    # Create a PyTorch tensor from the NumPy array
    target_tensor = torch.tensor(target_array, dtype=torch.long)  # Assuming target is of type long/int

    # Create a TensorDataset
    dataset = TensorDataset(data_tensor, target_tensor)

    # Define batch size
    batch_size = 1

    # Define number of DataLoader workers
    num_workers = 0  # Set this to a positive integer to enable multi-process data loading

    # Create DataLoader
    test_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

    # Assuming net_model is an instance of your model
    predictions = net_model.predict(test_loader)
    # predictions = np.array(predictions)
    # predictions_flat = predictions.flatten()

    return predictions

In [94]:
predict_binary_model(pretrained_binary_test_embeddings, binary_y_test)

Accuracy: 0.49166666666666664


0.49166666666666664

### Ternary Classification

In [95]:
N_ternary_embeddings, d_ternary_embeddings = my_ternary_train_embeddings.shape

In [96]:
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 3
dropout_rate = 0.2

net_model = Net(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)
print(net_model)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [97]:
def train_ternary_model(ternary_train_embeddings, ternary_train_y):
    ternary_train_y = ternary_train_y.replace(2, 0)
    ternary_train_y = ternary_train_y.replace(3, 2)
    data_tensor = torch.tensor(ternary_train_embeddings, dtype=torch.float32)
    target_tensor = torch.tensor(ternary_train_y.values, dtype=torch.long)
    dataset = TensorDataset(data_tensor, target_tensor)

    # how many samples per batch to load
    batch_size = 64

    # as a positive integer will turn on multi-process data loading with the specified number of loader worker processes; otherwise, single-process data loading
    num_workers = 0

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

    number_of_epochs = 3
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net_model.parameters(), lr=0.01)
    output_of_model = net_model.train_network(number_of_epochs, optimizer, criterion, train_loader)
    
    return output_of_model

In [98]:
train_ternary_model(pretrained_ternary_train_embeddings, ternary_y_train)

Epoch: 1 	Training Loss: 1.071023
Epoch: 2 	Training Loss: 1.069538
Epoch: 3 	Training Loss: 1.068447


In [99]:
def predict_ternary_model(ternary_test_embeddings, ternary_test_y):
    ternary_test_y = ternary_test_y.replace(2, 0)
    ternary_test_y = ternary_test_y.replace(3, 2)
    target_array = ternary_test_y.values
    data_tensor = torch.tensor(ternary_test_embeddings, dtype=torch.float32)


    # Create a PyTorch tensor from the NumPy array
    target_tensor = torch.tensor(target_array, dtype=torch.long)  # Assuming target is of type long/int

    # Create a TensorDataset
    dataset = TensorDataset(data_tensor, target_tensor)

    # Define batch size
    batch_size = 1

    # Define number of DataLoader workers
    num_workers = 0  # Set this to a positive integer to enable multi-process data loading

    # Create DataLoader
    test_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

    # Assuming net_model is an instance of your model
    predictions = net_model.predict(test_loader)
    # predictions = np.array(predictions)
    # predictions_flat = predictions.flatten()

    return predictions

In [100]:
predict_ternary_model(pretrained_ternary_test_embeddings, ternary_y_test)

Accuracy: 0.4033333333333333


0.4033333333333333